<a href="https://colab.research.google.com/github/PMarrhia02/HealthCare/blob/main/HealthCare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install findspark

In [ ]:
import pyspark
import pandas as pd
import plotly.express as px


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving healthcare_dataset.csv to healthcare_dataset.csv


In [ ]:
import pandas as pd
pd.read_csv('healthcare_dataset.csv')

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55495,eLIZABeTH jaCkSOn,42,Female,O+,Asthma,2020-08-16,Joshua Jarvis,Jones-Thompson,Blue Cross,2650.714952,417,Elective,2020-09-15,Penicillin,Abnormal
55496,KYle pEREz,61,Female,AB-,Obesity,2020-01-23,Taylor Sullivan,Tucker-Moyer,Cigna,31457.797307,316,Elective,2020-02-01,Aspirin,Normal
55497,HEATher WaNG,38,Female,B+,Hypertension,2020-07-13,Joe Jacobs DVM,"and Mahoney Johnson Vasquez,",UnitedHealthcare,27620.764717,347,Urgent,2020-08-10,Ibuprofen,Abnormal
55498,JENniFER JOneS,43,Male,O-,Arthritis,2019-05-25,Kimberly Curry,"Jackson Todd and Castro,",Medicare,32451.092358,321,Elective,2019-05-31,Ibuprofen,Abnormal


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Practice").getOrCreate()

In [ ]:
spark

In [ ]:
df = spark.read.csv('healthcare_dataset.csv')

In [ ]:
df.head()

Row(_c0='Name', _c1='Age', _c2='Gender', _c3='Blood Type', _c4='Medical Condition', _c5='Date of Admission', _c6='Doctor', _c7='Hospital', _c8='Insurance Provider', _c9='Billing Amount', _c10='Room Number', _c11='Admission Type', _c12='Discharge Date', _c13='Medication', _c14='Test Results')

In [ ]:
df = spark.read.option('header','true').csv('healthcare_dataset.csv',inferSchema=True)

In [ ]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Blood Type: string (nullable = true)
 |-- Medical Condition: string (nullable = true)
 |-- Date of Admission: date (nullable = true)
 |-- Doctor: string (nullable = true)
 |-- Hospital: string (nullable = true)
 |-- Insurance Provider: string (nullable = true)
 |-- Billing Amount: double (nullable = true)
 |-- Room Number: integer (nullable = true)
 |-- Admission Type: string (nullable = true)
 |-- Discharge Date: date (nullable = true)
 |-- Medication: string (nullable = true)
 |-- Test Results: string (nullable = true)



In [ ]:
df.dtypes

[('Name', 'string'),
 ('Age', 'int'),
 ('Gender', 'string'),
 ('Blood Type', 'string'),
 ('Medical Condition', 'string'),
 ('Date of Admission', 'date'),
 ('Doctor', 'string'),
 ('Hospital', 'string'),
 ('Insurance Provider', 'string'),
 ('Billing Amount', 'double'),
 ('Room Number', 'int'),
 ('Admission Type', 'string'),
 ('Discharge Date', 'date'),
 ('Medication', 'string'),
 ('Test Results', 'string')]

In [ ]:
df.describe()

DataFrame[summary: string, Name: string, Age: string, Gender: string, Blood Type: string, Medical Condition: string, Doctor: string, Hospital: string, Insurance Provider: string, Billing Amount: string, Room Number: string, Admission Type: string, Medication: string, Test Results: string]

In [ ]:
px.pie(df, names='Blood Type', title='Distribution of Blood Group')


In [ ]:
df = df.drop('Name', 'Doctor', 'Room Number')

In [ ]:
from pyspark.sql.functions import datediff

df = df.withColumn("Length of Stay", datediff("Discharge Date", "Date of Admission"))
df.show()


+---+------+----------+-----------------+-----------------+--------------------+------------------+------------------+--------------+--------------+-----------+------------+--------------+
|Age|Gender|Blood Type|Medical Condition|Date of Admission|            Hospital|Insurance Provider|    Billing Amount|Admission Type|Discharge Date| Medication|Test Results|Length of Stay|
+---+------+----------+-----------------+-----------------+--------------------+------------------+------------------+--------------+--------------+-----------+------------+--------------+
| 30|  Male|        B-|           Cancer|       2024-01-31|     Sons and Miller|        Blue Cross|18856.281305978155|        Urgent|    2024-02-02|Paracetamol|      Normal|             2|
| 62|  Male|        A+|          Obesity|       2019-08-20|             Kim Inc|          Medicare|33643.327286577885|     Emergency|    2019-08-26|  Ibuprofen|Inconclusive|             6|
| 76|Female|        A-|          Obesity|       2022-09

In [ ]:
df.coalesce(1).write.csv('cleaned_health-care-single', header=True, mode='overwrite')


In [ ]:
!zip -r output_csv_single.zip cleaned_health-care-single
files.download("output_csv_single.zip")



  adding: cleaned_health-care-single/ (stored 0%)
  adding: cleaned_health-care-single/_SUCCESS (stored 0%)
  adding: cleaned_health-care-single/part-00000-fd11d57c-7eed-4a89-a332-d17e27329df0-c000.csv (deflated 71%)
  adding: cleaned_health-care-single/._SUCCESS.crc (stored 0%)
  adding: cleaned_health-care-single/.part-00000-fd11d57c-7eed-4a89-a332-d17e27329df0-c000.csv.crc (deflated 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>